In [108]:
from scapy.utils import RawPcapReader
from scapy.layers.l2 import Ether
from scapy.layers.inet import IP, TCP

from tqdm.notebook import tqdm

In [109]:
from enum import Enum

In [110]:
class PktDirection(Enum):
    not_defined = 0
    client_to_server = 1
    server_to_client = 2

In [111]:
def process_pcap(pcap_file_in):
    pkts = {}
    client_port = '2222'
    server_port = '2223'
    times = []
    resols = []
    sizes = []
    ip_srcs = []
    ip_dsts = []
    for (pkt_data, pkt_metadata,) in RawPcapReader(pcap_file_in):
        ether_pkt = Ether(pkt_data)
        if 'type' not in ether_pkt.fields:
            continue
        if ether_pkt.type != 0x0800:
            continue
        ip_pkt = ether_pkt[IP]
        if ip_pkt.proto != 6:
            continue
        tcp_pkt = ip_pkt[TCP]
        if (tcp_pkt.sport not in (int(client_port), int(server_port))) and (tcp_pkt.dport not in (int(client_port), int(server_port))):
            continue
        last_pkt_timestamp = (pkt_metadata.tshigh << 32) | pkt_metadata.tslow
        last_pkt_timestamp_resolution = pkt_metadata.tsresol
        tcp_payload_len = ip_pkt.len - (ip_pkt.ihl * 4) - (tcp_pkt.dataofs * 4)
        pkt_data = {}
        times.append(last_pkt_timestamp)
        resols.append(last_pkt_timestamp_resolution)
        sizes.append(tcp_payload_len)
        ip_srcs.append(ip_pkt.src)
        ip_dsts.append(ip_pkt.dst)
    pkts = {'timestamp': times, 'val': sizes, 'resol': resols, 'src': ip_srcs, 'dst': ip_dsts}
    return pkts

In [112]:
pp = process_pcap('data/node-1/capture-output.pcap')

In [113]:
import pandas as pd

In [114]:
df = pd.DataFrame(pp).to_csv('tshark.csv', index=False)